<a href="https://colab.research.google.com/github/Kiet2000/ActionDetectionforSignLanguage/blob/main/Code_with_precision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import time
start_time = time.time()


In [105]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from IPython.display import display
from random import shuffle
from itertools import combinations

1. Load the data with encoding
Concatenate the values in each record into one single string.
2. Change all alphabetical characters into lowercase.
3. Convert multiple spaces to one.
4. Combine the records from both tables into one big list as we did during the lab but keep the ID's intact.

In [106]:

# Load the datasets
df_ACM = pd.read_csv('ACM.csv', header=0, quotechar='"', sep=",", na_values=['na', '-', '.', ''])
df_DBLP2 = pd.read_csv('DBLP2.csv', header=0, quotechar='"', sep=",", encoding="ISO-8859-1", na_values=['na', '-', '.', ''])

# Concatenate the values in each record into one single string
df_ACM['combined'] = df_ACM.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
df_DBLP2['combined'] = df_DBLP2.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Change all alphabetical characters into lowercase and convert multiple spaces to one
df_ACM['combined'] = df_ACM['combined'].str.lower().str.replace(r'\s+', ' ', regex=True)
df_DBLP2['combined'] = df_DBLP2['combined'].str.lower().str.replace(r'\s+', ' ', regex=True)

# Combine the records from both tables into one list while keeping the IDs
combined_data_with_id = list(zip(df_ACM['id'], df_ACM['combined'])) + list(zip(df_DBLP2['id'], df_DBLP2['combined']))



 compute the shingles, the minhash signature and the similarity.

In [107]:
def shingle(text, k=5):
    """Create a set of k-shingles from the input text."""
    return {text[i:i + k] for i in range(len(text) - k + 1)}

shingled_list = [shingle(text) for _, text in combined_data_with_id]


In [108]:
def build_vocab(shingle_sets):
    """Build vocabulary from a list of shingle sets."""
    full_set = {shingle for s_set in shingle_sets for shingle in s_set}
    return {shingle: idx for idx, shingle in enumerate(full_set)}

def one_hot(shingles, vocab):
    """Generate one-hot encoding for a set of shingles based on the vocabulary."""
    vec = [0] * len(vocab)
    for shingle in shingles:
        idx = vocab.get(shingle)
        if idx is not None:
            vec[idx] = 1
    return vec

vocab = build_vocab(shingled_list)
document_shingle_matrix = [one_hot(s, vocab) for s in shingled_list]


In [109]:
def get_minhash_arr(num_hashes:int,vocab:dict):
    """
    Generates a MinHash array for the given vocabulary.

    This function creates an array where each row represents a hash function and
    each column corresponds to a word in the vocabulary. The values are permutations
    of integers representing the hashed value of each word for that particular hash function.

    Parameters:
    - num_hashes (int): The number of hash functions (rows) to generate for the MinHash array.
    - vocab (dict): The vocabulary where keys are words and values can be any data
      (only keys are used in this function).

    Returns:
    - np.ndarray: The generated MinHash array with `num_hashes` rows and columns equal
      to the size of the vocabulary. Each cell contains the hashed value of the corresponding
      word for the respective hash function.

    Example:
    vocab = {'apple': 1, 'banana': 2}
    get_minhash_arr(2, vocab)
    # Possible output:
    # array([[1, 2],
    #        [2, 1]])
    """
    length = len(vocab.keys())
    arr = np.zeros((num_hashes,length))
    for i in range(num_hashes):
        permutation = np.random.permutation(len(vocab.keys())) + 1
        arr[i,:] = permutation.copy()
    return arr.astype(int)
def get_signature(minhash:np.ndarray, vector:np.ndarray):
    """
    Computes the signature of a given vector using the provided MinHash matrix.

    The function finds the nonzero indices of the vector, extracts the corresponding
    columns from the MinHash matrix, and computes the signature as the minimum value
    across those columns for each row of the MinHash matrix.

    Parameters:
    - minhash (np.ndarray): The MinHash matrix where each column represents a shingle
      and each row represents a hash function.
    - vector (np.ndarray): A vector representing the presence (non-zero values) or
      absence (zero values) of shingles.

    Returns:
    - np.ndarray: The signature vector derived from the MinHash matrix for the provided vector.

    Example:
    minhash = np.array([[2, 3, 4], [5, 6, 7], [8, 9, 10]])
    vector = np.array([0, 1, 0])
    get_signature(minhash, vector)
    output:array([3, 6, 9])
    """
    idx = np.nonzero(vector)[0].tolist()
    shingles = minhash[:,idx]
    signature = np.min(shingles,axis=1)
    return signature

def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    return intersection_size / union_size if union_size != 0 else 0.0

def compute_signature_similarity(signature_1, signature_2):
    """
    Calculate the similarity between two signature matrices using MinHash.

    Parameters:
    - signature_1: First signature matrix as a numpy array.
    - signature_matrix2: Second signature matrix as a numpy array.

    Returns:
    - Estimated Jaccard similarity.
    """
    # Ensure the matrices have the same shape
    if signature_1.shape != signature_2.shape:
        raise ValueError("Both signature matrices must have the same shape.")
    # Count the number of rows where the two matrices agree
    agreement_count = np.sum(signature_1 == signature_2)
    # Calculate the similarity
    similarity = agreement_count / signature_2.shape[0]

    return similarity

In [110]:
#LSH funtcion:

class LSH:
    """
    Implements the Locality Sensitive Hashing (LSH) technique for approximate
    nearest neighbor search.
    """
    buckets = []
    counter = 0

    def __init__(self, b: int):
        """
        Initializes the LSH instance with a specified number of bands.

        Parameters:
        - b (int): The number of bands to divide the signature into.
        """
        self.b = b
        for i in range(b):
            self.buckets.append({})

    def make_subvecs(self, signature: np.ndarray) -> np.ndarray:
        """
        Divides a given signature into subvectors based on the number of bands.

        Parameters:
        - signature (np.ndarray): The MinHash signature to be divided.

        Returns:
        - np.ndarray: A stacked array where each row is a subvector of the signature.
        """
        l = len(signature)
        assert l % self.b == 0
        r = int(l / self.b)
        subvecs = []
        for i in range(0, l, r):
            subvecs.append(signature[i:i+r])
        return np.stack(subvecs)

    def add_hash(self, signature: np.ndarray):
        """
        Adds a signature to the appropriate LSH buckets based on its subvectors.

        Parameters:
        - signature (np.ndarray): The MinHash signature to be hashed and added.
        """
        subvecs = self.make_subvecs(signature).astype(str)
        for i, subvec in enumerate(subvecs):
            subvec = ','.join(subvec)
            if subvec not in self.buckets[i].keys():
                self.buckets[i][subvec] = []
            self.buckets[i][subvec].append(self.counter)
        self.counter += 1

    def check_candidates(self) -> set:
        """
        Identifies candidate pairs from the LSH buckets that could be potential near duplicates.

        Returns:
        - set: A set of tuple pairs representing the indices of candidate signatures.
        """
        candidates = []
        for bucket_band in self.buckets:
            keys = bucket_band.keys()
            for bucket in keys:
                hits = bucket_band[bucket]
                if len(hits) > 1:
                    candidates.extend(combinations(hits, 2))
        return set(candidates)

In [111]:
# Step 1: Generate the MinHash Array
num_hashes = 500  # The number of hash functions to use for MinHash
minhash_array = get_minhash_arr(num_hashes, vocab)

# Step 2: Compute MinHash Signatures for all documents
signatures = [get_signature(minhash_array, vec) for vec in document_shingle_matrix]

# Step 3: Compute Similarities (for demonstration, we'll compute similarity between the first two documents)
# Estimated Jaccard similarity using MinHash signatures
similarity_estimate = compute_signature_similarity(signatures[0], signatures[1])

# Actual Jaccard similarity between the original shingle sets
actual_similarity = jaccard_similarity(shingled_list[0], shingled_list[1])

print(f"Estimated Jaccard Similarity: {similarity_estimate}")
print(f"Actual Jaccard Similarity: {actual_similarity}")


Estimated Jaccard Similarity: 0.226
Actual Jaccard Similarity: 0.20502092050209206


Use the functions in the tutorials from lab 5 to compute the shingles, the minhash signature and the similarity.

In [112]:
# Number of bands for LSH
bands = 50

# Initialize LSH
lsh = LSH(bands)

# Add all signatures to LSH
for signature in signatures:
    lsh.add_hash(signature)

# Extract candidate pairs using LSH
candidates = lsh.check_candidates()

# Get top 2224 candidates
top_candidates = list(candidates)[:2224]

In [113]:
df_PerfectMapping = pd.read_csv('DBLP-ACM_perfectMapping.csv', header=0, quotechar='"', sep=",", na_values=['na', '-', '.', ''])


Extract the top 2224 candidates from the LSH algorithm, compare them to the actual mappings in the file DBLP-ACM_perfectMapping.csv and compute the precision of the method.

In [114]:

actual_mappings = set(tuple(rec) for rec in df_PerfectMapping[['idACM', 'idDBLP']].to_records(index=False))
# First, determine the split index where ACM data ends and DBLP2 data begins
split_index = len(df_ACM)

# Filter the candidates to ensure that one ID is from ACM and the other is from DBLP2
filtered_candidates = [(i, j) for i, j in candidates if i < split_index and j >= split_index]

# Get the top 2224 candidates from the filtered list
top_candidates_filtered = filtered_candidates[:2224]

# Construct top_candidates_ids_adjusted using the filtered candidate pairs
top_candidates_ids_adjusted = {(combined_data_with_id[i][0], combined_data_with_id[j][0]) for i, j in top_candidates_filtered}

# Compute precision using the adjusted top_candidates_ids
correct_candidates_adjusted = actual_mappings.intersection(top_candidates_ids_adjusted)
precision_adjusted = len(correct_candidates_adjusted) / len(top_candidates_ids_adjusted)

print(f"Adjusted Precision: {precision_adjusted:.4f}")
print("Correctly identified pairs:", correct_candidates_adjusted)
print(len(correct_candidates_adjusted))



Adjusted Precision: 0.9320
Correctly identified pairs: {(672213, 'conf/vldb/NarayananS01'), (253274, 'conf/sigmod/HoAMS97'), (223857, 'conf/sigmod/Team95'), (273254, 'journals/sigmod/BurtonM98'), (191930, 'conf/sigmod/DanielsDDEHJJLSSS94'), (362091, 'journals/sigmod/KantM00'), (564695, 'conf/sigmod/KalnisNOPT02'), (637428, 'journals/sigmod/Grohe02'), (181567, 'journals/sigmod/NorrieBSW94'), (212020, 'journals/sigmod/RosenthalSMBTL95'), (671869, 'conf/vldb/HelalL00'), (604271, 'journals/sigmod/SayginVC01'), (233333, 'conf/sigmod/HarinarayanRU96'), (640993, 'journals/sigmod/Chang03'), (245906, 'journals/sigmod/Sidell96'), (604282, 'journals/sigmod/ShahMFH01'), (672008, 'conf/vldb/ChoenniV00'), (671495, 'conf/vldb/TamuraK99'), (672039, 'conf/vldb/SindoniTABFGMPPT01'), (306144, 'journals/sigmod/EisenbergM98a'), (671524, 'conf/vldb/CeriFP99'), (564750, 'conf/sigmod/DogacTPPLKTK02'), (672375, 'conf/vldb/ChaHKK01'), (671517, 'conf/vldb/JermaineDO99'), (603886, 'journals/sigmod/LabrinidisM01')

Record the running time of the method.

In [115]:
end_time = time.time()
total_time = end_time - start_time
print(f"Total running time: {total_time:.2f} seconds")



Total running time: 79.33 seconds
